In [ ]:
from miniai import def_device, to_device

In [ ]:
def_device

In [ ]:
to_device??

In [ ]:
from utils import load_sklearn_dataset

import torch
from torch import nn
from torch.utils.data import DataLoader

from miniai import (
    Dataset,
    DataLoaders,
    Learner,
    TrainCB,
    MetricsCB,
    DeviceCB,
    ProgressCB,
)


class RegressionNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load_sklearn_dataset(
    "california_housing"
)

In [ ]:
# Create Dataset instances
train_ds = Dataset(X_train, y_train)
valid_ds = Dataset(X_val, y_val)
test_ds = Dataset(X_test, y_test)

# Prepare DataLoaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=64)

# Initialize the neural network
input_size = X_train.shape[1]  # Features count
output_size = 1  # Single regression output
model = RegressionNN(input_size, output_size)

# Prepare the learner
loss_func = nn.MSELoss()
lr = 0.01
opt_func = torch.optim.Adam

cbs = [
    TrainCB(),  # Handles the core steps in the training loop. Can be left out if using TrainLearner
    DeviceCB(),  # Handles making sure data and model are on the right device
    MetricsCB(),  # Keep track of any relevant metrics
    ProgressCB(),  # Displays metrics and loss during training, optionally plot=True for a pretty graph
]

learner = Learner(
    model, DataLoaders(train_dl, val_dl), loss_func, lr, opt_func=opt_func, cbs=cbs
)

# Train the model
learner.fit(n_epochs=10, train=True, valid=True)

# Optionally, evaluate the model on the test set if required
test_dl = DataLoader(test_ds, batch_size=64)
# Further testing and evaluation logic here

In [ ]:
import math, typing
from collections.abc import Mapping
from copy import copy
from itertools import zip_longest
from functools import partial, wraps
from operator import attrgetter, itemgetter

import matplotlib.pyplot as plt
import numpy as np
import fastcore.all as fc
from fastprogress import progress_bar, master_bar

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import default_collate

from torcheval.metrics import Mean
from einops import rearrange
from accelerate import Accelerator

from fastcore.test import *
import operator
import pandas as pd

In [ ]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
print(df)

In [ ]:
test([1, 2], [1, 2], operator.eq)

In [ ]:
def _fail():
    raise Exception()


test_fail(_fail)

In [ ]:
test([1, 2], [1, 2], operator.eq)
test_fail(lambda: test([1, 2], [1], operator.eq))

In [ ]:
# test(["abc"], ["abc"], all_equal)
# test_fail(lambda: test(["abc"], ["cab"], all_equal))

In [ ]:
def _fail_args(a):
    print(a)
    if a == 5:
        raise ValueError


test_fail(_fail_args, args=(5,))
test_fail(_fail_args, kwargs=dict(a=5))

In [ ]:
# _fail_args(5)

In [ ]:
# ifnone(a, b)

In [ ]:
# test_eq(ifnone(None, 1), 1)
# test_eq(ifnone(2, 1), 2)

In [ ]:
from fastcore.all import *


class MyClass:
    def __init__(self, a, b, c=3):
        store_attr()  # Automatically assigns a, b, c as attributes


obj = MyClass(1, 2)
print(obj.a)  # Output: 1
print(obj.b)  # Output: 2
print(obj.c)  # Output: 3


class Animal:
    def __init__(self, name):
        store_attr()

    def make_sound(self, sound):
        print(f"{self.name} says {sound}")


# Use fastcore's patch to add new methods to Animal
@patch
def sit(self: Animal):
    print(f"{self.name} sits down.")


@patch
def roll_over(self: Animal):
    print(f"{self.name} rolls over.")


dog = Animal("Buddy")
dog.make_sound("Woof!")
dog.sit()
dog.roll_over()


class BaseModel:
    def __init__(self, input_size, output_size, activation):
        store_attr()
        print(
            f"Initialized BaseModel with input_size={input_size}, output_size={output_size}, activation={activation}"
        )


@delegates(BaseModel.__init__)  # Delegate parameters from BaseModel.__init__
class AdvancedModel(BaseModel):
    def __init__(self, hidden_layers, **kwargs):
        super().__init__(**kwargs)
        self.hidden_layers = hidden_layers
        print(f"AdvancedModel adds hidden_layers={self.hidden_layers}")


model = AdvancedModel(
    input_size=784, output_size=10, activation="relu", hidden_layers=[128, 64]
)

In [ ]:
AdvancedModel??

In [ ]:
class Base:
    def __init__(self, a, b):
        pass


@delegates(Base.__init__)
class Derived(Base):
    def __init__(self, c, d, **kwargs):
        super().__init__(**kwargs)

In [ ]:
Derived??

In [ ]:
import inspect

In [ ]:
print(inspect.signature(Derived.__init__))